In [1]:
# disable warnings
import warnings
warnings.filterwarnings("ignore")

# basic imports
import tensorflow as tf 
import torch
import io
import numpy as np
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

from keras.utils import dataset_utils
import matplotlib.pyplot as plt

import baseline_config

In [2]:
def paths_and_labels_to_dataset(image_paths,labels,num_classes):
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    img_ds = path_ds.map(
        lambda path: tf.io.read_file(path), 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    label_ds = dataset_utils.labels_to_dataset(
        labels, 
        'categorical', 
        num_classes)
    img_ds = tf.data.Dataset.zip((img_ds, label_ds))
    return img_ds

def create_dataset(subset):
    image_paths, labels, class_names = dataset_utils.index_directory(
            baseline_config.dataset_path + subset,
            labels="inferred",
            formats=('.pt'),
            class_names=None,
            shuffle=False,
            seed=42,
            follow_links=False)

    dataset = paths_and_labels_to_dataset(
        image_paths=image_paths,
        labels=labels,
        num_classes=len(class_names))
    
    return dataset, class_names

train_dataset, class_names = create_dataset('TRAIN/')
test_dataset, _            = create_dataset('TEST/')
validation_dataset, _      = create_dataset('VALIDATION/')
print("class names: ", class_names)

Found 12384 files belonging to 5 classes.
Found 487 files belonging to 5 classes.
Found 384 files belonging to 5 classes.
class names:  ['brant', 'jabwar', 'sheowl', 'spodov', 'wiltur']


In [3]:
def dataset_tranforms(image,label):
    image = tf.io.parse_tensor(image, tf.float32)
    #image = torch.load(io.BytesIO(image.numpy()))
    image = tf.expand_dims(image, -1)
    image = tf.repeat(image, 3, 2)
    # label = tf.cast(label, tf.int64)
   
    #print("shape is ", label.shape)
    #print("image is ", image.shape)
   
    #assert label.shape == (5,) 
    #assert image.shape == (313, 128, 3)
    
    return image,label

#py_func_wrapper = lambda x,y: tf.py_function(func=dataset_tranforms, inp=[x,y], Tout=[tf.float32,tf.float32])

train_dataset_b = ( 
                  train_dataset
                  .shuffle(20000)
                  .map(dataset_tranforms)
                  .batch(baseline_config.batch_size)
                  .repeat()            
                )

validation_dataset_b = ( 
                  validation_dataset
                  .map(dataset_tranforms)
                  .batch(baseline_config.batch_size)
                )

test_dataset_b = ( 
                  test_dataset
                  .map(dataset_tranforms)
                  .batch(baseline_config.batch_size)
                )

In [4]:
for item,lbl in train_dataset_b.take(1):
    print(item.shape, lbl.shape)

(16, 313, 128, 3) (16, 5)


In [5]:
# build a really simple classification model using a pre-training Efficientnet V2
model = keras.Sequential(
    [
        # use the model as a feature generator only
        # need to resize here, as the efficientnet_v2_imagenet1k_b3 model requires 260x260 input
        #tf.keras.layers.Resizing(260, 260, interpolation="bilinear", crop_to_aspect_ratio=False),
        #hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/feature_vector/2", False),
        
        # use the model as a feature generator only
        # need to resize here, as the efficientnet_v2_imagenet21k_s model expects it
        tf.keras.layers.InputLayer(input_shape=(313,128,3))
        tf.keras.layers.Resizing(384, 384, interpolation="bilinear", crop_to_aspect_ratio=False),
        hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2", False),        

        # add the classification layer here       
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.70),
        layers.Dense(len(class_names), activation=None),
    ]
)
# need to tell the model what the input shape is
model.build([None, 313, 128, 3])

# show the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 384, 384, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              20331360  
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 20,660,581
Trainable params: 329,221
Non-t

In [6]:
# the form_logits means the loss function has the 'softmax' buillt in.  This approach is numerically more stable
# than including the softmax activation on the last layer of the classifier
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=baseline_config.learning_rate), 
              metrics=["accuracy"],
              )

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True)

model.fit(train_dataset_b, 
          validation_data=validation_dataset_b,
          steps_per_epoch=50,
          callbacks=[model_checkpoint_callback],
          epochs=baseline_config.max_epoch)

Epoch 1/100000


ValueError: in user code:

    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Andrew\miniconda3\envs\dev\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "sequential" "                 f"(type Sequential).
    
    